## 資料集準備

資料集與 Challenge 1 相同，使用 Kaggle 的 `mlg-ulb/creditcardfraud`，移除 `Time` 欄位並將 `Amount` 標準化。



In [34]:
import kagglehub
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

# general setting. do not change TEST_SIZE
RANDOM_SEED = 42
TEST_SIZE = 0.3

# load data
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")
data = pd.read_csv(f"{path}/creditcard.csv")
data["Class"] = data["Class"].astype(int)
data = data.drop(["Time"], axis=1)
data["Amount"] = StandardScaler().fit_transform(data["Amount"].values.reshape(-1, 1))

## Hybrid Model

Baseline:

```
Hybrid Mode Evaluation:
=============================================
         Accuracy: 0.9996722961506501
  Precision Score: 0.9285714285714286
     Recall Score: 0.8602941176470589
         F1 Score: 0.8931297709923665

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85307
           1       0.93      0.86      0.89       136

    accuracy                           1.00     85443
   macro avg       0.96      0.93      0.95     85443
weighted avg       1.00      1.00      1.00     85443
```

### 資料處理

首先拆分每筆資料的所有特徵 `X` 與對應的 label `y`，並依照 `TEST_SIZE` 分別拆分成訓練跟測試兩部分。

原本有嘗試做 PCA 嘗試提升準確度，但效果反而更差。推測是因為資料集本身已經做過 PCA 了，再做一次變化不大，參數沒調好而造成反效果。

In [35]:
# split data
X = np.asarray(data.drop(columns=["Class"]))
y = np.asarray(data["Class"])
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_SEED
)

### Isolation Forest

Isolation Forest 可以找出一堆資料中的異常值，很適合用在這個資料集。

參數設定：

- `contamination`: 預期有多少比例的異常值，設為整個資料集的詐騙占比 0.17%。
- `random_state`: 設置隨機種子，讓相同參數下的實驗結果一致。
- `n_estimators`: 模型要建立多少棵樹來預測，設一個較大的值 300。
- `bootstrap`: 讓模型使用會放回的重複抽樣 (Bootstrap Method) 建立訓練過程的子樣本，以增加數的多樣性，讓模型更穩健。
- `n_jobs`: 設為 -1 使用所有 CPU 核心加速計算。

---

訓練完成後將預測結果作為**新的特徵**加到資料集，將非監督式模型的結果提供給監督式模型參考。

In [39]:
isolation = IsolationForest(
    contamination=0.0017,
    random_state=RANDOM_SEED,
    n_estimators=300,
    bootstrap=True,
    n_jobs=-1
)
isolation.fit(X_train)

# use all data to predict
iso_labels = isolation.predict(X_train)
iso_labels = (iso_labels == -1).astype(int)

# combine to dataset as a new feature
X_train = np.hstack([X_train, iso_labels.reshape(-1, 1)])
iso_pred_test = isolation.predict(X_test)
iso_feature_test = (iso_pred_test == -1).astype(int)
X_test = np.hstack((X_test, iso_feature_test.reshape(-1, 1)))

### XGBoost

監督式學習的部分使用跟 Challenge 1 一樣的 XGBoost，但是使用了加入非監督式模型結果的訓練資料。

參數設置：

- `random_state`: 設置隨機種子，讓相同參數下的實驗結果一致。
- `enable_categorical`: 使用分類模式。
- `n_estimators`: 經過多組參數測試，設置 300 的效果最好，設更高結果不再提升。
- `tree_method`: 分類模式需要使用 `approx` 或 `hist` 演算法，前者兼顧效率與準確度。
- `device`: 使用 GPU 加速計算。
- `learning_rate`: 預設值是 0.3，在 40 步之後開始出現 overfitting 的現象，
- `n_jobs`: -1 表示用所有 CPU 核心進行平行計算。

### 結果

Recall 比 baseline 低，其餘指標皆有提升：

|      指標       |   Baseline   |   My Model   |
|:---------------:|:------------:|:------------:|
|    Accuracy     |   0.999672   | **0.999661** |
| Precision Score |   0.928571   | **0.949580** |
|  Recall Score   | **0.860294** |   0.830882   |
|    F1 Score     |   0.893130   | **0.886275** |

In [48]:
xgb = XGBClassifier(
    random_state=RANDOM_SEED,
    enable_categorical=True,
    n_estimators=350,
    tree_method="approx",
    device="cuda",
    learning_rate=0.1,
    n_jobs=-1,
)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)


def evaluation(y_true, y_pred, model_name="Model"):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print(f"\n{model_name} Evaluation:")
    print("===" * 15)
    print("         Accuracy:", accuracy)
    print("  Precision Score:", precision)
    print("     Recall Score:", recall)
    print("         F1 Score:", f1)
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))


evaluation(y_test, y_pred, model_name="Isolation Forest + XGBoost")


Isolation Forest + XGBoost Evaluation:
         Accuracy: 0.9996488887328394
  Precision Score: 0.9344262295081968
     Recall Score: 0.8382352941176471
         F1 Score: 0.8837209302325582

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85307
           1       0.93      0.84      0.88       136

    accuracy                           1.00     85443
   macro avg       0.97      0.92      0.94     85443
weighted avg       1.00      1.00      1.00     85443

